In [1]:
pip install -q transformers accelerate fastapi uvicorn pyngrok

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "bigscience/mt0-large"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto",early_stopping=True)

inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt").to("cuda")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> I love you.</s>


In [33]:
from fastapi import FastAPI

app = FastAPI()

In [34]:
@app.get("/translate_text/")
async def translate(user_id:int,text:str,og_lang:str,translated_lang:str):
  original_language = og_lang
  encoded_input = tokenizer.encode(f"Translate to {translated_lang}: {text}.", return_tensors="pt").to("cuda")
  output = model.generate(encoded_input)
  translated_text = tokenizer.decode(output[0])

  return {"original_text":text,"original_language":original_language,"translated_language":translated_lang,"transated_text":translated_text}

In [35]:
@app.get("/")
async def translate():
  return {"message":"hello"}

In [36]:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)
ngrok_tunnel

<NgrokTunnel: "https://c484-34-125-10-196.ngrok.io" -> "http://localhost:8000">

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [5551]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     27.107.113.146:0 - "GET / HTTP/1.1" 200 OK
INFO:     27.107.113.146:0 - "GET /translate_text/?user_id=1212&text=hello%20there%20my%20name%20is%20viraj&translated_lang=French&og_lang=english HTTP/1.1" 200 OK
